In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn import model_selection

input_file = './data/income_data.txt'

Xy = []
count_class1 = 0
count_class2== 0
max_datapoints = 25000

with open(input_file, 'r') as f:
    for line in f.readlines():
        if count_class1 >= max_datapoints and count_class2 >= max_datapoints:
            break
            
        if '?' in line:
            continue
            
            